In [1]:
import os, sys
import torch
import numpy as np
import argparse
from argparse import Namespace
import tqdm
import six
from scipy import stats
import pickle
from helper import set_seeds
from torch.utils.data import DataLoader
from datasets.datasets import get_scaled_data, get_synthetic_data
from utils.q_model_ens import QModelEns, MSEModel
from losses import batch_qr_loss, batch_interval_loss
import helper
from helper import SYN_DATA, REAL_DATA

sys.modules['sklearn.externals.six'] = six
np.warnings.filterwarnings('ignore')

os.environ["MKL_CBWR"] = 'AUTO'

results_path = helper.results_path

if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"


def get_loss_fn(loss_name):
    if loss_name == 'batch_qr' or loss_name == 'batch_wqr':
        fn = batch_qr_loss
    elif loss_name == 'batch_int':
        fn = batch_interval_loss
    else:
        raise ValueError('loss arg not valid')

    return fn


def parse_args():
    parser = argparse.ArgumentParser()

    parser.add_argument('--seed', type=int, default=None,
                        help='random seed')

    parser.add_argument('--seed_begin', type=int, default=None,
                        help='random seed')
    parser.add_argument('--seed_end', type=int, default=None,
                        help='random seed')

    parser.add_argument('--data', type=str, default='',
                        help='dataset to use')

    parser.add_argument('--num_q', type=int, default=30,
                        help='number of quantiles you want to sample each step')
    parser.add_argument('--gpu', type=int, default=1,
                        help='gpu num to use')

    parser.add_argument('--num_ep', type=int, default=10000,
                        help='number of epochs')
    parser.add_argument('--nl', type=int, default=2,
                        help='number of layers')
    parser.add_argument('--hs', type=int, default=64,
                        help='hidden size')

    parser.add_argument('--dropout', type=float, default=0,
                        help='dropout ratio of the dropout level')
    parser.add_argument('--lr', type=float, default=1e-3,
                        help='learning rate')
    parser.add_argument('--wd', type=float, default=0.0,
                        help='weight decay')
    parser.add_argument('--bs', type=int, default=1024,
                        help='batch size')
    parser.add_argument('--wait', type=int, default=200,
                        help='how long to wait for lower validation loss')

    parser.add_argument('--loss', type=str,
                        help='specify type of loss')

    parser.add_argument('--corr_mult', type=float, default=0.,
                        help='correlation penalty multiplier')

    parser.add_argument('--hsic_mult', type=float, default=0.,
                        help='correlation penalty multiplier')

    parser.add_argument('--ds_type', type=str, default="",
                        help='type of data set. real or synthetic. REAL for real. SYN for synthetic')

    parser.add_argument('--test_ratio', type=float, default=0.4,
                        help='ratio of test set size')

    parser.add_argument('--save_training_results', type=int, default=0,
                        help='1 for saving results during training, or 0 for not saving')

    parser.add_argument('--method', type=str, default='QR',
                        help='method to use (QR or qr_forest)')

    args = parser.parse_args()

    os.environ['CUDA_VISIBLE_DEVICES'] = str(args.gpu)
    device_name = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device_name)
    args.device = device
    if args.method not in ['QR', 'qr_forest']:
        raise ValueError('method arg not valid')

    return args


def update_results_during_training(y_upper, y_lower, x, y, set_name, results_dict, alpha):
    with torch.no_grad():
        if len(x) == 0 or len(y) == 0:
            return
        y = y.reshape(-1).to(device)
        idx = np.random.permutation(len(x))  # [:len(xx)]
        x = x[idx].to(device)
        quantiles = torch.Tensor([alpha / 2, 1 - alpha / 2]).to(device)
        

        if torch.is_tensor(y):
            curr_y = y.cpu().detach().numpy()[idx]
        else:
            curr_y = y[idx]
        in_the_range = ((curr_y >= y_lower) & (curr_y <= y_upper))
        lengths = (y_upper - y_lower)

        if 'pearsons_correlation' + '_over_' + set_name not in results_dict:
            results_dict['pearsons_correlation' + '_over_' + set_name] = []

        results_dict['pearsons_correlation' + '_over_' + set_name] += [
            stats.pearsonr(in_the_range, lengths)[0]]

        if 'coverage' + '_over_' + set_name not in results_dict:
            results_dict['coverage' + '_over_' + set_name] = []

        results_dict['coverage' + '_over_' + set_name] += [np.mean(in_the_range)]

        if 'interval_lengths' + '_over_' + set_name not in results_dict:
            results_dict['interval_lengths' + '_over_' + set_name] = []

        results_dict['interval_lengths' + '_over_' + set_name] += [np.mean(lengths)]


In [ ]:
from utils.penalty_multipliers import real_corr_per_dataset_per_loss
from Experiments.EXP1.trainer import model_callByName, loss_callByName

POSSIBLE_REAL_DATA_NAMES = ['kin8nm', 'naval', 'meps_19', 'meps_20', 'meps_21', 'facebook_1', 'facebook_2',
                            'blog_data', 'bio', 'scaled_bio', 'bike']


data_type = REAL_DATA

DATA_NAMES = ['meps_19', 'meps_20', 'meps_21', 'facebook_1', 'facebook_2', 'blog_data']



SEEDS = range(0, 1)

save_results_during_training = True


arg_loss = 'batch_int'
TRAINING_OVER_ALL_QUANTILES = True

    

for d in DATA_NAMES:
    
    print("On dataset "+ d)

    for s in tqdm.tqdm(SEEDS):

        arg_data = d
        arg_seed = s
        
        
        set_seeds(arg_seed)
        data_args = Namespace(dataset=arg_data, seed=arg_seed)

        # if data_type == REAL_DATA:
        # Fetching data
        data_out = get_scaled_data(arg_data, arg_seed, recal_prop = 0.1)
        x_train, y_train = data_out.x_train, data_out.y_train
        unscaled_x_train = None
        unscaled_x_test = None
        minority_group_uncertainty = None
        group_feature = None


        x_tr, x_va, x_te, y_tr, y_va, y_te, y_al = \
            data_out.x_tr, data_out.x_va, data_out.x_te, data_out.y_tr, \
            data_out.y_va, data_out.y_te, data_out.y_al
        
        x_va, y_va = x_va.to(device), y_va.to(device)
        x_tr, y_tr = x_tr.to(device), y_tr.to(device)
        y_te, x_te = y_te.to(device), x_te.to(device)
        x_train, y_train = x_train.to(device), y_train.to(device)
        x_test = x_te
        y_test = y_te

        w_tr, w_va, get_tr_weights = helper.get_wqr_weights(arg_loss, x_tr, y_tr, x_va, y_va, device = device)
        
        y_range = (y_al.max() - y_al.min()).item()

        # creating the model
        num_tr = x_tr.shape[0]
        dim_x = x_tr.shape[1]
        dim_y = y_tr.shape[1]

        model_ens = QModelEns(input_size=dim_x + 1, output_size=dim_y,
                              hidden_size=64, num_layers=2, dropout=0,
                              lr=1e-3, wd=0,
                              num_ens=1, device=device)

        loader = DataLoader(helper.IndexedDataset(x_tr, y_tr),
                            shuffle=True,
                            batch_size=1024)

        # Loss function
        loss_fn = get_loss_fn(arg_loss)
        batch_loss = True if 'batch' in arg_loss else False
        """ train loop """
        tr_loss_list = []
        va_loss_list = []
        te_loss_list = []
        batch_size = 1024
        for ep in range(10000):

            if model_ens.done_training:
                print('Done training ens at EP {}'.format(ep))
                break

            # Take train step
            ep_train_loss = []  # list of losses from each batch, for one epoch
            epoch_loss = []

            for xi, yi, index in loader:
                if TRAINING_OVER_ALL_QUANTILES:
                    q_list = torch.rand(30)
                else:
                    q_list = torch.Tensor([alpha / 2])
                    
                arg_corr_mult = real_corr_per_dataset_per_loss["int"][d]
                loss = model_ens.loss(loss_fn, xi, yi, q_list,
                                      batch_q=batch_loss,
                                      take_step=True, args=arg_corr_mult, weights=get_tr_weights(index))
                ep_train_loss.append(loss)

            ep_tr_loss = np.nanmean(np.stack(ep_train_loss, axis=0), axis=0)
            tr_loss_list.append(ep_tr_loss)
            
            

            # Validation loss
            # x_va, y_va = x_va.to(args.device), y_va.to(args.device)
            if TRAINING_OVER_ALL_QUANTILES:
                va_te_q_list = torch.linspace(0.01, 0.99, 99)
            else:
                va_te_q_list = torch.Tensor([alpha / 2, 1 - alpha / 2])

            ep_va_loss = model_ens.update_va_loss(
                loss_fn, x_va, y_va, va_te_q_list,
                batch_q=batch_loss, curr_ep=ep, num_wait=200,
                args=arg_corr_mult, weights=w_va
            )
            va_loss_list.append(ep_va_loss)

            # Printing some losses
            if (ep % 1000 == 0):
                print('EP:{}'.format(ep))
                pass
        # Move everything to cpu
        x_tr, y_tr, x_va, y_va, x_te, y_te = \
            x_tr.cpu(), y_tr.cpu(), x_va.cpu(), y_va.cpu(), x_te.cpu(), y_te.cpu()
        model_ens.use_device(torch.device('cpu'))

        quantiles = torch.Tensor(np.linspace(0.01,0.99,99))
        test_preds = model_ens.predict_q(
            x_te, quantiles, ens_pred_type='conf',
            recal_model=None, recal_type=None
        )
        test_preds.detach().cpu().numpy()

        # print(test_preds.shape)
        
        
        
        
        
        
        
        
        test_Y = y_te.view(-1)
        ret = {}
        val_criterias = [
                "MACE_Loss", "AGCE_Loss", "CheckScore"
            ]  

        for key in val_criterias:

            real_loss = loss_callByName[key]

            real_err = real_loss(torch.permute(test_preds, (1,0)), test_Y, q_list = np.linspace(0.01,0.99,99)).item()

            if isinstance(real_err, torch.Tensor):

                real_err = real_err.item()

            ret[key] = real_err


        print(ret)
        print("--------------------------------------------------")

    
        

On dataset meps_19


  0%|          | 0/1 [00:00<?, ?it/s]

EP:0


100%|██████████| 1/1 [01:18<00:00, 78.81s/it]


Done training ens at EP 563
{'MACE_Loss': 0.030589619651436806, 'AGCE_Loss': 0.05355070158839226, 'CheckScore': 0.11384144425392151}
--------------------------------------------------
On dataset meps_20


...

EP:0


100%|██████████| 1/1 [01:17<00:00, 77.33s/it]


Done training ens at EP 564
{'MACE_Loss': 0.04197763651609421, 'AGCE_Loss': 0.06319630146026611, 'CheckScore': 0.10958028584718704}
--------------------------------------------------
On dataset facebook_1


  0%|          | 0/1 [00:00<?, ?it/s]

EP:0
